```
    ██╗   ██╗ ██████╗ ██╗      █████╗  ██████╗████████╗
    ╚██╗ ██╔╝██╔═══██╗██║     ██╔══██╗██╔════╝╚══██╔══╝
     ╚████╔╝ ██║   ██║██║     ███████║██║        ██║   
      ╚██╔╝  ██║   ██║██║     ██╔══██║██║        ██║   
       ██║   ╚██████╔╝███████╗██║  ██║╚██████╗   ██║   
       ╚═╝    ╚═════╝ ╚══════╝╚═╝  ╚═╝ ╚═════╝   ╚═╝ 
```


In [67]:
from utils import VideoReaders, DetectorLoader
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
from skimage import segmentation
from itertools import groupby
import random, warnings
from SORT.sort import *
from skimage import measure, color, segmentation, draw
from scipy.spatial.distance import cdist
import seaborn as sns

# Step 1: Identify True Positives

In [75]:
ground_truths = pd.read_csv('datasets/slip-fall/mladen-holdout/slip-fall-mladen-ground-truth.csv')
dataset = []
for flag, df in ground_truths.groupby('Fall?'):
    pathlist = [os.path.join(
        '../datasets/slip-fall/mladen-holdout/05-FPS/0.25x/',
        x
    ) for x in df['File']]
    df['Paths'] = pathlist
    dataset.append(df)

dataset = pd.concat(dataset)

def check_video_length(x):
    '''
    Uses the VideoLoader to detect if the frames are too long for use in our dataset on a series of data in a dataframe. 
    '''
    frame_provider = VideoReaders.VideoReader(x)
    length, shape = frame_provider.properties()
    if length > 100:
        return "Too Big."
    else:
        return "Just Right."

dataset['Usage'] = dataset['Paths'].apply(check_video_length)
usable_data = dataset[(dataset['Usage'] == 'Just Right.')]


In [76]:
class ChainedAssignment:
    def __init__(self, chained=None):
        acceptable = [None, 'warn', 'raise']
        assert chained in acceptable, "chained must be in " + str(acceptable)
        self.swcw = chained

    def __enter__(self):
        self.saved_swcw = pd.options.mode.chained_assignment
        pd.options.mode.chained_assignment = self.swcw
        return self

    def __exit__(self, *args):
        pd.options.mode.chained_assignment = self.saved_swcw

def instantaneous_time(x):
    frame_interval = 0.2
    return (x[1] - x[0]) * frame_interval

def angular_displacement(x):
    return (x[1] - x[0])

# Step 2: Load input video


In [77]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    model = DetectorLoader.YOLACT('./weights/slip-fall-chosen-weights/yolact_resnet50_432_74900.pth', threshold = 0.25)

In [87]:
for path, _ in usable_data.groupby('Paths'):
    frame_provider = VideoReaders.VideoReader(path)
    length, shape = frame_provider.properties()
    print('{} is {} frames.'.format(os.path.basename(path),length))

    frames = []
    inference = []
    for frame in frame_provider:
        frames.append(frame[:,:,[2,1,0]])
        c, s, bb, ma = model.predict(frame[:,:,[2,1,0]])
        idx = np.where(c == 0) # Person has a Class ID of 0
        pixelwise_arrays = []
        for item in idx:
            for n, qq in enumerate(item):
                pixelwise = np.zeros_like(ma[qq,...])
                pixelwise = ma[qq,...].astype(np.uint8)
                pixelwise[np.where(pixelwise == 1)] = n + 1
                pixelwise_arrays.append(pixelwise)

        filtered = []
        for array in pixelwise_arrays:
            if np.max(array) != 0:
                filtered.append(array)

        stacked = np.sum(filtered, axis = 0) # Will cause the array pixel values to increase in numerical value. This also maintains multiple people in the frame. 
        inference.append(stacked)

    output = []
    for arry in inference: # Check if the shape coming out of the model is OK.
        if len(arry.shape) < 2:
            output.append(np.zeros(shape).astype(np.uint8))
        else:
            output.append(arry)

    def check(list1, val):
        return(any(x < val for x in list1))

    # Remove anything that's not in the frame
    for frame_position in range(0, length):
        generated_image = np.zeros_like(output[frame_position])
        for pixel in np.unique(output[frame_position]):
            if pixel == 0: 
                pass
            else:
                ignore_perimeter = 25 # Tunable parameter for ignore zone
                end_extent = (generated_image.shape[0] - ignore_perimeter, generated_image.shape[1] - ignore_perimeter)
                perim_rr, perim_cc = draw.rectangle_perimeter(start = (ignore_perimeter, ignore_perimeter), end = end_extent)
                rectangle_coords = np.stack((perim_rr, perim_cc), axis = 1)
                generated_image[rectangle_coords[:,0], rectangle_coords[:,1]] = 1 # Debugging
                for otherregion in measure.regionprops((output[frame_position] == pixel).astype(np.uint8)):
                    mask_coordinates = otherregion.coords
                    generated_image[mask_coordinates[:,0], mask_coordinates[:,1]] = 1
                distances = cdist(mask_coordinates, rectangle_coords, metric = 'euclidean') # Compute the euclidean distance between points and imposed boundary 
                minimum_distance = np.min(distances, axis = 1).tolist()
                integer_minimum_distances = [int(item) for item in minimum_distance]
                if check(integer_minimum_distances, 1):
                    output[frame_position][output[frame_position] == pixel] = 0

    # Derive measurement of angles
    minimal_dictionary = []
    for i in range(0, length):
        for region in measure.regionprops(output[i], color.rgb2gray(frames[i])):
            minimal_dictionary.append({
                "Frame": i, 
                "ID": region.label, 
                "BBOX": region.bbox
            })
    tracked_bboxes = []
    Sorter = Sort(max_age = 5, min_hits = 1, iou_threshold = 0.2) # 1 Tunable parameters for SORT
    for instance, entry in groupby(minimal_dictionary, key = lambda x:x['Frame']):
        entry_list = []
        for item in entry:
            lst = list(item['BBOX'])
            lst.extend([0, item['ID']])
            entry_list.append(lst)
        track_bbs_ids = Sorter.update(np.array(entry_list))
        for objects in track_bbs_ids:
            r0, c0, r1, c1, ID, label = objects.tolist()
            for region in measure.regionprops(output[instance], color.rgb2gray(frames[instance])):
                if region.label == label:
                    this_normalized_moment = region.moments_normalized
                    angle = (np.arctan2(2*this_normalized_moment[1,1], this_normalized_moment[2,0] - this_normalized_moment[0,2]))/2 # Normalized image moments 
                    w = region.bbox[2] - region.bbox[0]
                    h = region.bbox[3] - region.bbox[1]
                    ar = w / float(h)
                    tracked_bboxes.append({'Frame': instance, 
                                            'ID': int(label), 
                                            'Track ID': int(ID), 
                                            'BBOX': region.bbox,
                                            'θ': angle,
                                            'Area': region.area,
                                            'Aspect Ratio': ar, 
                                            'Eccentricity': region.eccentricity, 
                                            'Perimeter': region.perimeter})
    tracked_dataframe = pd.DataFrame(tracked_bboxes)

    super_output = []
    if len(tracked_dataframe) == 0:
        super_output.append({'File': None, 'Frame': None, 'Bounding Box': None, 'ID': None, 'θ': None, 'dθ': None, 'Δt': None, '⍵': None, '⍵^2': None})
    else:
        for tracks, track_df in tracked_dataframe.groupby('Track ID'):
            # TODO Might want to add an if statement to suppress tracks with length less than track age. 
            current_track = track_df.loc[(track_df['Track ID'] == tracks), :]
            with ChainedAssignment():
                current_track['dθ'] = current_track.loc[(current_track['Track ID'] == tracks), :]['θ'].rolling(window = 2).apply(angular_displacement, raw = True).fillna(0)
                current_track['Δt'] = current_track.loc[(current_track['Track ID'] == tracks), :]['Frame'].rolling(window = 2).apply(instantaneous_time, raw = True).fillna(0).cumsum()
                current_track['⍵'] = current_track.loc[(current_track['Track ID'] == tracks), :]['dθ'].diff() / current_track.loc[(current_track['Track ID'] == tracks), :]['Δt'].diff().fillna(0)
                current_track['⍵^2'] = current_track.loc[(current_track['Track ID'] == tracks), :]['⍵'].diff() / current_track.loc[(current_track['Track ID'] == tracks), :]['Δt'].diff()
                super_output.append(current_track)

    test = super_output[0]
    try:
        df = pd.melt(test[['ID', 'Frame', 'θ', 'dθ', 'Δt', '⍵', '⍵^2']], 
                id_vars = ['Frame'], 
                value_vars = ['dθ', '⍵', '⍵^2'],
                var_name = 'Parameter',
                value_name = 'Value')
        f,ax = plt.subplots(1)
        with sns.axes_style('whitegrid'):
            sns.despine()
            sns.lineplot(data = df, x = 'Frame', y = 'Value', hue = 'Parameter', ax = ax)
            ax.set(ylabel = 'θ (radians)', ylim = (-50, 50))
        f.tight_layout()
        f.savefig('Chart-{}'.format(os.path.basename(path).replace('.mp4', '.png')), dpi = 300)
        plt.close(f)
    except TypeError:
        print('{} did not produce a chart.'.format(os.path.basename(path)))
        continue

0009.mp4 is 84 frames.
0010.mp4 is 49 frames.
0011.mp4 is 90 frames.
0012.mp4 is 89 frames.
0013.mp4 is 91 frames.
0014.mp4 is 80 frames.
0015.mp4 is 75 frames.
0016.mp4 is 60 frames.
0017.mp4 is 63 frames.
0017.mp4 did not produce a chart.
0018.mp4 is 62 frames.
0019.mp4 is 75 frames.
0020.mp4 is 80 frames.
0021.mp4 is 60 frames.
0022.mp4 is 69 frames.
0023.mp4 is 55 frames.
0024.mp4 is 56 frames.
0025.mp4 is 59 frames.
0026.mp4 is 61 frames.
0028.mp4 is 43 frames.
0029.mp4 is 53 frames.
0030.mp4 is 38 frames.
0031.mp4 is 44 frames.
0032.mp4 is 24 frames.
0033.mp4 is 24 frames.
0034.mp4 is 23 frames.


In [54]:
# # Create segmentation boundary mask for printout
# if len(output) != len(frames):
#     raise ValueError("Frame Lengths are off.")
# boundaries = []
# for im, mask in zip(frames, output):
#     boundaries.append(segmentation.mark_boundaries(im, mask, mode = 'thick', color = (0,1,0)))

# # Create matplotlib. 
# columns = 8
# rows, r = divmod(length, columns)
# if r:
#     rows += 1
# f, ax = plt.subplots(rows, columns, figsize = (50,60))
# ax = ax.flatten()
# for n in range(0, length):
#     ax[n].imshow(boundaries[n], aspect = 'auto')
#     current_frame = str(n)
#     ax[n].set(title = 'Frame {}'.format(current_frame.zfill(4)))
#     ax[n].axis('off')
# f.tight_layout()
# f.savefig('{}.png'.format(os.path.basename(path).replace('.mp4', '')), dpi = 100)
# plt.close(f)